In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import tifffile as tiff
import os
import sys


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DIR_INPUT = '../input/hubmap-kidney-segmentation'
all_shapes = [(31262, 14844), (40429, 31295), (49780, 26840), (43780, 36800),
              (42360, 38160), (22165, 29433), (47723, 23990), (44066, 31299),
              (53240, 37040), (29020, 46660), (50680, 39960), (43160, 33240)]
all_shapes = sorted(all_shapes, key=lambda x: x[0])
public_test_names = ('b9a3865fc', 'b2dc8411c', '26dc41664', 'c68fe75ea', 'afa5e8098')

sub_df = pd.read_csv('{}/sample_submission.csv'.format(DIR_INPUT))

test_set_path = os.path.join(DIR_INPUT, 'test')
image_ids = list(set([os.path.splitext(name)[0]
                     for name in os.listdir(test_set_path)
                     if name.endswith('.tiff')]))
if sub_df.shape[0] > 5:
    online_shapes = []
    for i, row in sub_df.iterrows():
        image = tiff.imread('{}/test/{}.tiff'.format(DIR_INPUT, row['id']))
        image = np.squeeze(image)
        if image.shape[0] == 3:
            image = image.transpose([1, 2, 0])
            image = np.ascontiguousarray(image)
        h, w, _ = image.shape
        online_shapes.append((w, h))

    online_shapes = sorted(online_shapes, key=lambda x: x[0])
    is_shape_equal = [False for _ in range(len(online_shapes))]
    for i in range(len(online_shapes)):
        if all_shapes[0] == online_shapes[0] and all_shapes[1] == online_shapes[1]:
            is_shape_equal[i] = True
    # whether all online shapes are the same with given shapes
    if all(is_shape_equal):
        sub_df.to_csv('submission.csv', index=False)
    else:
        sys.exit()
else:
    # This is for committing only.
    sub_df.to_csv('submission.csv', index=False)
